In [1]:
import pandas as pd
from datetime import date

In [2]:
df = pd.read_json('../data/raw/StockSeries.json', orient='records')

In [3]:
all_dfs = []
cids = df['CID'].unique()

for cid in cids:
    filter_df = df[df['CID'] == cid].sort_values(by='Date', ascending=False)

    close_values = filter_df['Close'].values.tolist()
    ct = close_values[:-1]
    ct_1 = close_values[1: ]
    change_pct = [((x - y) / y) * 100 for x, y in zip(ct, ct_1)]

    filter_df = filter_df.iloc[:-1, :]
    filter_df['change_pct'] = change_pct
    
    all_dfs.append(filter_df)

stock_pct = pd.concat(all_dfs)

In [4]:
articles = pd.read_csv('../data/final/articles_with_llm_outputs.csv')
articles['Date'] = pd.to_datetime(articles['Date'])

In [5]:
dates = stock_pct[stock_pct['CID'] == 1]['Date']
dates = pd.DataFrame(dates)

In [6]:
def find_closest_date(rowDate):
    dates['difference'] = dates['Date'][dates['Date'] >= rowDate] - rowDate
    return dates.sort_values(by='difference').iloc[0, 0]

In [7]:
# articles[~articles['Date'].isin(final['Date'])]

articles.loc[~articles['Date'].isin(stock_pct['Date']), 'Date'] = articles.loc[~articles['Date'].isin(stock_pct['Date']), 'Date'].apply(find_closest_date)

In [9]:
final = articles.merge(stock_pct.loc[:, ['CID', 'Date', 'change_pct']], on=['CID', 'Date'])
final['impact_score'] = final['sentiment'] * final['change_pct']

In [11]:
final.to_csv('../data/final/articles_with_llm_outputs_and_impact_score.csv', index=False)
final.to_json('../data/final/articles_with_llm_outputs_and_impact_score.json', orient='records')

In [19]:
final[['CID', 'impact_score']].groupby(by='CID').mean()

,impact_score
CID,
1,-0.012710
2,0.857826
3,0.029327
4,0.208765
9,0.181322
10,0.250989
11,0.285720
12,-0.278601
13,-0.080474


In [21]:
final[final['CID'] == 1].sort_values(by='impact_score', ascending=True)['impact_score'].describe()

count    367.000000
mean      -0.012710
std        1.638085
min       -6.172505
25%       -0.857637
50%        0.036620
75%        0.866402
max        6.172505
Name: impact_score, dtype: float64